In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/catastici.db")

In [2]:
import pandas as pd
questions = pd.read_csv("test_data.csv")
questions.head()

,question_id,level_len,level_nest,question,true_query,true_answer
0,0,0,0,What are the various kinds of properties ident...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
1,0,0,0,Could you enumerate the different property cat...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
2,0,0,0,"In the dataset, what property types can be found?","SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
3,0,0,0,Specify all the property types that are encaps...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
4,0,0,0,List all the types of properties in the dataset.,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."


## Build Matched Contents

In [3]:
import re
def get_evidence(question, db = db, fields = ["Owner_ID","Owner_First_Name","Owner_Family_Name","Property_Type","Rent_Income","Property_Location"]):
    keywords = re.findall(r'"(.*?)"', question)
    if len(keywords) == 0:
        return "None"
    evidence = []
    for keyword in keywords:
        for field in fields:
            if len(db.run(f"""SELECT * FROM catastici WHERE "{field}" = "{keyword}" LIMIT 1;""")) > 0:
                evidence.append(f'"{field}" = "{keyword}"')
                break
    return "\n".join(evidence)

# example
print(questions['question'][360])
print(get_evidence(questions['question'][360]))

What’s the minimum amount "zuane" "panizza" earns from the property at "traghetto di san silvestro"?
"Owner_First_Name" = "zuane"
"Owner_Family_Name" = "panizza"
"Property_Location" = "traghetto di san silvestro"


In [4]:
# Build evidence
evidences = []
for i, r in questions.iterrows():
    evidences.append(get_evidence(r['question']))
questions['matched_contents'] = evidences
questions.sample(5)

,question_id,level_len,level_nest,question,true_query,true_answer,matched_contents
375,75,0,1,What is the count of properties classified und...,"SELECT ""Property_Type"", COUNT(""Property_Type"")...","[('casa a pepian', 501, 1.0728369076324267)]","""Property_Type"" = ""casa a pepian"""
38,7,0,0,Could you identify the lowest rent income valu...,"SELECT MIN(""Rent_Income"") AS Lowest_Rent_Incom...","[(1,)]",None
266,53,0,0,Can you enumerate the different kinds of prope...,"SELECT ""Property_Location"", COUNT(DISTINCT ""Pr...",[('[appartamento] li formenti sopra la strada ...,None
265,53,0,0,What is the count of property varieties that a...,"SELECT ""Property_Location"", COUNT(DISTINCT ""Pr...",[('[appartamento] li formenti sopra la strada ...,None
489,97,1,1,What is the average number of properties owned...,"SELECT ""Owner_First_Name"", ""Owner_Family_Name""...","[('_consorte', 'vianoli', 1.0), ('_eredita', '...",None


In [5]:
questions.to_csv("test_data_with_evidence.csv", index=False)